# Часть 1. Vanilla Autoencoder (10 баллов)

## 1.1. Подготовка данных (0.5 балла)


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os

from torchvision import datasets, transforms

import torch.nn as nn
from torch.utils.data import Dataset
import torch

from PIL import Image

%matplotlib inline

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [3]:
torch.cuda.is_available()

True

In [4]:
RESCALE_SIZE = 128

In [5]:
df_attrs = pd.read_csv("lfw_attributes.txt",sep='\t',skiprows=1,) 
df_attrs = pd.DataFrame(df_attrs.iloc[:,:-1].values, columns = df_attrs.columns[1:])

#read photos
photo_ids = []
for dirpath, dirnames, filenames in os.walk('lfw-deepfunneled'):
    for fname in filenames:
        if fname.endswith(".jpg"):
            fpath = os.path.join(dirpath,fname)
            photo_id = fname[:-4].replace('_',' ').split()
            person_id = ' '.join(photo_id[:-1])
            photo_number = int(photo_id[-1])
            photo_ids.append({'person':person_id,'imagenum':photo_number,'photo_path':fpath})

photo_ids = pd.DataFrame(photo_ids)

df = pd.merge(df_attrs,photo_ids,on=('person','imagenum'))

all_attrs = df.drop(["photo_path","person","imagenum"],axis=1)

In [6]:
class CustomDataSet(Dataset):
    def __init__(self, paths, transform):
        self.transform = transform
        self.paths = paths

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_loc = self.paths[idx]
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

In [7]:
transform = {
            'train': transforms.Compose([
                transforms.CenterCrop(size=(RESCALE_SIZE, RESCALE_SIZE)),
                transforms.RandomRotation(degrees=10),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor()
            ]),
            'val_test': transforms.Compose([
                transforms.Resize(size=(RESCALE_SIZE, RESCALE_SIZE)),
                transforms.ToTensor()
            ]),
        }


Разбейте выборку картинок на train и val, выведите несколько картинок в output, чтобы посмотреть, как они выглядят, и приведите картинки к тензорам pytorch, чтобы можно было скормить их сети:

In [8]:
paths = df.photo_path.values

my_dataset = CustomDataSet(paths, transform=transform['train'])

In [9]:
val = 0.2

val_size = int(len(my_dataset) * val)

train_set, val_set = torch.utils.data.random_split(my_dataset, [len(my_dataset) - val_size, val_size])

In [10]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size)

## 1.2. Архитектура модели (1.5 балла)
В этом разделе мы напишем и обучем обычный автоэнкодер.



<img src="https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F4b8adf79-8e6a-4b7d-9061-8617a00edbb1%2F__2021-04-30__14.53.33.png?table=block&id=56f187b4-279f-4208-b1ed-4bda5f91bfc0&width=2880&userId=3b1b5e32-1cfb-4b0f-8705-5a524a8f56e3&cache=v2" alt="Autoencoder">


^ напомню, что автоэнкодер выглядит вот так

In [11]:
dim_code = 256 # выберите размер латентного вектора

Реализуем autoencoder. Архитектуру (conv, fully-connected, ReLu, etc) можете выбирать сами. Экспериментируйте!

In [12]:
class Autoencoder(nn.Module):
    def __init__(self, in_channels, latent_dim):
        super(Autoencoder, self).__init__()
                
        modules = []
        hidden_dims = [32, 64, 128, 256, 512, 1024]

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size=3, stride=2, padding=1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*4, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*4, latent_dim)
        
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * 4)

        hidden_dims.reverse()

        for i in range(len(hidden_dims)-1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i+1],
                                       kernel_size=3,
                                       stride=2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i+1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)
        
        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels=3,
                                      kernel_size=3, padding=1),
                            nn.Tanh())
        
    def encode(self, input):
        result = self.encoder(input)
        
        result = torch.flatten(result, start_dim=1)
        
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return mu, log_var
    
    def decode(self, z):
        result = self.decoder_input(z)
        result = result.view(-1, 1024, 2, 2)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu
 
    
    def get_latent_vector(self, mu, log_var):
        z = self.reparameterize(mu, log_var)
        return z
 
    def forward(self, input):
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        
        reconstruction = torch.sigmoid(self.decode(z))
        
        return reconstruction, mu, log_var

In [13]:
def KL_divergence(mu, logsigma):
    """
    часть функции потерь, которая отвечает за "близость" латентных представлений разных людей
    """
    loss = -0.5 * torch.sum(1 + logsigma - mu.pow(2) - logsigma.exp())
    return loss

def log_likelihood(x, reconstruction):
    """
    часть функции потерь, которая отвечает за качество реконструкции
    """
    loss = nn.BCELoss(reduction='sum')
    return loss(reconstruction, x)

def loss_vae(x, mu, logsigma, reconstruction):
    return KL_divergence(mu, logsigma) + log_likelihood(x, reconstruction)

criterion = loss_vae

autoencoder = Autoencoder(3, dim_code).to(device)

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0003)

In [14]:
device

'cuda:0'

## 1.3 Обучение (2 балла)

Осталось написать код обучения автоэнкодера. При этом было бы неплохо в процессе иногда смотреть, как автоэнкодер реконструирует изображения на данном этапе обучения. Наример, после каждой эпохи (прогона train выборки через автоэекодер) можно смотреть, какие реконструкции получились для каких-то изображений val выборки.

А, ну еще было бы неплохо выводить графики train и val лоссов в процессе тренировки =)

In [15]:
from tqdm.notebook import trange, tqdm_notebook

n_epochs = 10
train_losses = []
val_losses = []

for epoch in trange(n_epochs):
    autoencoder.train()
    train_losses_per_epoch = []
    for batch in tqdm_notebook(train_loader):
        batch = batch.float().to(device)
        optimizer.zero_grad()
        reconstruction, mu, log_var = autoencoder(batch)
        loss = criterion(batch, mu, log_var, reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = [] 
    with torch.no_grad():
        for batch in val_loader:
            batch = batch.float().to(device)
            reconstruction, mu, log_var = autoencoder(batch)
            loss = criterion(batch, mu, log_var, reconstruction)
            val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.arange(len(train_losses)), train_losses, label='Train')
plt.plot(np.arange(len(val_losses)), val_losses, label='Validation')

plt.xlabel('Epoch')
plt.title('MSE loss')
plt.legend()
plt.show()

Давайте посмотрим, как наш тренированный автоэнкодер кодирует и восстанавливает картинки:

In [ ]:
autoencoder.eval()
with torch.no_grad():
  reconstruction, mu, log_var = autoencoder(batch.to(device))
  reconstruction = reconstruction
  result = reconstruction.cpu().detach().numpy().transpose(0, 2, 3, 1)
  ground_truth = batch.cpu().numpy().transpose(0, 2, 3, 1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 20))
for i, (gt, res) in enumerate(zip(ground_truth[:5], result[:5])):
  plt.subplot(5, 2, 2*i+1)
  plt.imshow(gt)
  plt.subplot(5, 2, 2*i+2)
  plt.imshow(res)

Not bad, right? 

## 1.4. Sampling (2 балла)

Давайте теперь будем не просто брать картинку, прогонять ее через автоэекодер и получать реконструкцию, а попробуем создать что-то НОВОЕ

Давайте возьмем и подсунем декодеру какие-нибудь сгенерированные нами векторы (например, из нормального распределения) и посмотрим на результат реконструкции декодера:

__Подсказка:__Е сли вместо лиц у вас выводится непонятно что, попробуйте посмотреть, как выглядят латентные векторы картинок из датасета. Так как в обучении нейронных сетей есть определенная доля рандома, векторы латентного слоя могут быть распределены НЕ как `np.random.randn(25, <latent_space_dim>)`. А чтобы у нас получались лица при запихивании вектора декодеру, вектор должен быть распределен так же, как латентные векторы реальных фоток. Так что в таком случае придется рандом немного подогнать.

In [ ]:
# сгенерируем 25 рандомных векторов размера latent_space
z = np.random.normal(loc=0, scale=1, size=(25, dim_code))
#z = np.random.randn(25, dim_code)
output = autoencoder.decode(torch.FloatTensor(z).to(device))

output = output.permute(0, 2, 3, 1).cpu().detach().numpy()

plt.figure(figsize=(18, 18))
for i in range(output.shape[0]):
  plt.subplot(output.shape[0] // 5, 5, i + 1)
  generated = output[i]
  plt.imshow((generated * 255).astype(np.uint8))

## Time to make fun! (4 балла)

Давайте научимся пририсовывать людям улыбки =)

<img src="https://i.imgur.com/tOE9rDK.png" alt="linear" width="700" height="400">

План такой:

1. Нужно выделить "вектор улыбки": для этого нужно из выборки изображений найти несколько (~15) людей с улыбками и столько же без.

Найти людей с улыбками вам поможет файл с описанием датасета, скачанный вместе с датасетом. В нем указаны имена картинок и присутствубщие атрибуты (улыбки, очки...)

2. Вычислить латентный вектор для всех улыбающихся людей (прогнать их через encoder) и то же для всех грустненьких

3. Вычислить, собственно, вектор улыбки -- посчитать разность между средним латентным вектором улыбающихся людей и средним латентным вектором грустных людей

4. А теперь приделаем улыбку грустному человеку: добавим полученный в пункте 3 вектор к латентному вектору грустного человека и прогоним полученный вектор через decoder. Получим того же человека, но уже не грустненького!

In [ ]:
plt.figure(figsize=(18, 18))
smiling = all_attrs.index[all_attrs['Smiling'] > 2.5][:25].values

for idx, i in enumerate(smiling):
  plt.subplot(5, 5, idx + 1)
  generated = my_dataset[i].permute(1, 2, 0)
  plt.imshow(generated)

In [ ]:
plt.figure(figsize=(18, 18))
sad = all_attrs.index[all_attrs['Frowning'] > 2.5][:25].values
for idx, i in enumerate(sad):
  plt.subplot(5, 5, idx + 1)
  generated = my_dataset[i].permute(1, 2, 0)
  plt.imshow(generated)

In [ ]:
sum_smile = torch.zeros_like(torch.zeros((1, dim_code))).to(device)

for smile_idx in smiling:
    smiling_single = my_dataset[smile_idx].unsqueeze(0).to(device)

    latent = autoencoder.encode(smiling_single)

    reconstructed = autoencoder.get_latent_vector(*latent)

    sum_smile += reconstructed

smile_vector = sum_smile.mean(0)

In [ ]:
sum_sad = torch.zeros_like(torch.zeros((1, dim_code))).to(device)

for sad_idx in sad:
    sad_single = my_dataset[sad_idx].unsqueeze(0).to(device)

    latent = autoencoder.encode(sad_single)

    reconstructed = autoencoder.get_latent_vector(*latent)

    sum_sad += reconstructed

sad_vector = sum_sad.mean(0)

In [ ]:
len([my_dataset[sad_idx] for sad_idx in sad])

In [ ]:
sad_dataset = Dataset([my_dataset[sad_idx] for sad_idx in sad])

In [ ]:
delta = smile_vector - sad_vector

In [ ]:
autoencoder.eval()
with torch.no_grad():
    reconstruction, mu, log_var = autoencoder(sad_dataset.permute(0, 2, 3, 1))
    latent = autoencoder.get_latent_vector(mu, log_var) + delta
    
    reconstruction_smile = autoencoder.decode(latent)
    reconstruction_smile = reconstruction_smile.view(-1, 64, 64, 3)
    reconstruction_smile = reconstruction_smile.cpu().detach().numpy()
    
    reconstruction = reconstruction.view(-1, 64, 64, 3)
    result = reconstruction.cpu().detach().numpy()
    
    ground_truth = sad_dataset.cpu().numpy().transpose(0, 2, 3, 1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 50))
for i, (gt, res, sm) in enumerate(zip(ground_truth[:5], result[:5], reconstruction_smile[:5])):
  plt.subplot(25, 3, 3*i+1)
  plt.axis('off')
  plt.imshow(gt, interpolation='nearest')
  
  plt.subplot(25, 3, 3*i+2)
  plt.axis('off')
  plt.imshow(res, interpolation='nearest')
  
  plt.subplot(25, 3, 3*i+3)
  plt.axis('off')
  plt.imshow(sm, interpolation='nearest')

In [ ]:
output = autoencoder.decode(delta.unsqueeze(0).to(device))

plt.figure(figsize=(8, 8))
for i in range(output.shape[0]):
  plt.subplot(1, 1, i + 1)
  generated = output[i].cpu().detach().numpy().transpose(1, 2, 0)
  plt.imshow(generated)

### Это вектор улыбки (decoded) &#8593; &#8593; &#8593;

Вуаля! Вы восхитительны!

Теперь вы можете пририсовывать людям не только улыбки, но и много чего другого -- закрывать/открывать глаза, пририсовывать очки... в общем, все, на что хватит фантазии и на что есть атрибуты в `all_attrs` :)

# Часть 2: Variational Autoencoder (10 баллов) 

Займемся обучением вариационных автоэнкодеров — проапгрейженной версии AE. Обучать будем на датасете MNIST, содержащем написанные от руки цифры от 0 до 9

In [ ]:
batch_size = 1024
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## 2.1 Архитектура модели и обучение (2 балла)

Реализуем VAE. Архитектуру (conv, fully-connected, ReLu, etc) можете выбирать сами. Рекомендуем пользоваться более сложными моделями, чем та, что была на семинаре :) Экспериментируйте!

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
    
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(7*7*32, 64),
            nn.ReLU(),
            nn.Linear(64, dim_code * 2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(dim_code, 64),
            nn.ReLU(),
            nn.Linear(64, 32*7*7),
            nn.Unflatten(1, torch.Size([32, 7, 7])),
            nn.Upsample(scale_factor=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 1, kernel_size=3, padding=1)
        )

    def encode(self, x):
        x = self.encoder(x).view(-1, 2, dim_code)
        
        mu = x[:, 0, :]
        logsigma = x[:, 1, :]
        
        return mu, logsigma
    
    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            std = torch.exp(0.5 * logsigma)
            eps = torch.randn_like(std)
            sample = mu + (eps * std)
            return sample
        else:
            # на инференсе возвращаем не случайный вектор из нормального распределения, а центральный -- mu. 
            # на инференсе выход автоэнкодера должен быть детерминирован.
            return mu
    
    def decode(self, z):       
        x = self.decoder(z)
        
        return x

    def forward(self, x):
        mu, logsigma = self.encode(x)
        
        latent = self.gaussian_sampler(mu, logsigma)
            
        reconstruction = self.decode(latent)
        reconstruction = torch.sigmoid(reconstruction)
        
        return reconstruction, mu, logsigma

Определим лосс и его компоненты для VAE:

Надеюсь, вы уже прочитали материал в towardsdatascience (или еще где-то) про VAE и знаете, что лосс у VAE состоит из двух частей: KL и log-likelihood.

Общий лосс будет выглядеть так:

$$\mathcal{L} = -D_{KL}(q_{\phi}(z|x)||p(z)) + \log p_{\theta}(x|z)$$

Формула для KL-дивергенции:

$$D_{KL} = -\frac{1}{2}\sum_{i=1}^{dimZ}(1+log(\sigma_i^2)-\mu_i^2-\sigma_i^2)$$

В качестве log-likelihood возьмем привычную нам кросс-энтропию.

In [ ]:
def KL_divergence(mu, logsigma):
    loss = -0.5 * torch.sum(1 + logsigma - mu.pow(2) - logsigma.exp())
    return loss

def log_likelihood(x, reconstruction):
    loss = nn.BCELoss(reduction='sum')
    return loss(reconstruction, x)

def loss_vae(x, mu, logsigma, reconstruction):
    return KL_divergence(mu, logsigma) + log_likelihood(x, reconstruction)

И обучим модель:

In [ ]:
criterion = loss_vae

autoencoder = VAE().to(device)

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0003)

In [ ]:
from tqdm.notebook import trange, tqdm

n_epochs = 100
train_losses = []
val_losses = []

for epoch in trange(n_epochs):
    autoencoder.train()
    train_losses_per_epoch = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        reconstruction, mu, logsigma = autoencoder(X_batch.to(device))
        reconstruction = reconstruction.view(-1, 1, 28, 28)
        loss = criterion(X_batch.to(device).float(), mu, logsigma, reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
          reconstruction, mu, logsigma = autoencoder(X_batch.to(device))
          reconstruction = reconstruction.view(-1, 1, 28, 28)
          loss = criterion(X_batch.to(device).float(), mu, logsigma, reconstruction)
          val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.arange(len(train_losses)), train_losses, label='Train')
plt.plot(np.arange(len(val_losses)), val_losses, label='Validation')

plt.xlabel('Epoch')
plt.title('MSE loss')
plt.legend()
plt.show()

Давайте посмотрим, как наш тренированный VAE кодирует и восстанавливает картинки:

In [ ]:
autoencoder.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        reconstruction, mu, logsigma = autoencoder(X_batch.to(device))
        reconstruction = reconstruction.view(-1, 28, 28, 1)
        result = reconstruction.cpu().detach().numpy()
        ground_truth = X_batch.numpy().transpose(0, 2, 3, 1)
        break

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 20))
for i, (gt, res) in enumerate(zip(ground_truth[:5], result[:5])):
  plt.subplot(5, 2, 2*i+1)
  plt.imshow(gt)
  plt.subplot(5, 2, 2*i+2)
  plt.imshow(res)

Давайте попробуем проделать для VAE то же, что и с обычным автоэнкодером -- подсунуть decoder'у из VAE случайные векторы из нормального распределения и посмотреть, какие картинки получаются:

In [ ]:
# вспомните про замечание из этого же пункта обычного AE про распределение латентных переменных
z = np.array([np.random.normal(0, 1, 100) for i in range(10)])
output = <скормите z декодеру>
<выведите тут полученные картинки>

## 2.2. Latent Representation (2 балла)

Давайте посмотрим, как латентные векторы картинок цифр выглядят в пространстве.
Ваша задача -- изобразить латентные векторы картинок точками в двумерном просторанстве. 

Это позволит оценить, насколько плотно распределены латентные векторы изображений цифр в пространстве. 

Плюс давайте сделаем такую вещь: покрасим точки, которые соответствуют картинкам каждой цифры, в свой отдельный цвет

Подсказка: красить -- это просто =) У plt.scatter есть параметр c (color), см. в документации.


Итак, план:
1. Получить латентные представления картинок тестового датасета
2. С помощтю `TSNE` (есть в `sklearn`) сжать эти представления до размерности 2 (чтобы можно было их визуализировать точками в пространстве)
3. Визуализировать полученные двумерные представления с помощью `matplotlib.scatter`, покрасить разными цветами точки, соответствующие картинкам разных цифр.

In [ ]:
X, y = [], []

autoencoder.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
      reconstruction, mu, logsigma = autoencoder(X_batch.to(device))
      
      X.extend(autoencoder.gaussian_sampler(mu, logsigma).detach().cpu().numpy())
      y.extend(y_batch.detach().cpu().numpy())
      
      reconstruction = reconstruction.view(-1, 28, 28, 1)
      result = reconstruction.cpu().detach().numpy()
      ground_truth = X_batch.numpy().transpose(0, 2, 3, 1)

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

tsne = TSNE()

X_embedded = tsne.fit_transform(X)

palette = sns.color_palette("bright", 10)

In [ ]:
sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], hue=y, legend='full', palette=palette);

Что вы думаете о виде латентного представления?

__Congrats v2.0!__

## 2.3. Conditional VAE (6 баллов)


Мы уже научились обучать обычный AE на датасете картинок и получать новые картинки, используя генерацию шума и декодер. 
Давайте теперь допустим, что мы обучили AE на датасете MNIST и теперь хотим генерировать новые картинки с числами с помощью декодера (как выше мы генерили рандомные лица). 
И вот нам понадобилось сгенерировать цифру 8, и мы подставляем разные варианты шума, но восьмерка никак не генерится:(

Хотелось бы добавить к нашему AE функцию "выдай мне рандомное число из вот этого вот класса", где классов десять (цифры от 0 до 9 образуют десять классов).  Conditional AE — так называется вид автоэнкодера, который предоставляет такую возможность. Ну, название "conditional" уже говорит само за себя.

И в этой части задания мы научимся такие обучать.

### Архитектура

На картинке ниже представлена архитектура простого Conditional VAE.

По сути, единственное отличие от обычного -- это то, что мы вместе с картинкой в первом слое энкодера и декодера передаем еще информацию о классе картинки. 

То есть, в первый (входной) слой энкодера подается конкатенация картинки и информации о классе (например, вектора из девяти нулей и одной единицы). В первый слой декодера подается конкатенация латентного вектора и информации о классе.


![alt text](https://sun9-63.userapi.com/impg/Mh1akf7mfpNoprrSWsPOouazSmTPMazYYF49Tw/djoHNw_9KVA.jpg?size=1175x642&quality=96&sign=e88baec5f9bb91c8443fba31dcf0a4df&type=album)

![alt text](https://sun9-73.userapi.com/impg/UDuloLNKhzTBYAKewgxke5-YPsAKyGOqA-qCRg/MnyCavJidxM.jpg?size=1229x651&quality=96&sign=f2d21bfacc1c5755b76868dc4cfef39c&type=album)



На всякий случай: это VAE, то есть, latent у него все еще состоит из mu и sigma

Таким образом, при генерации новой рандомной картинки мы должны будем передать декодеру сконкатенированные латентный вектор и класс картинки.

P.S. Также можно передавать класс картинки не только в первый слой, но и в каждый слой сети. То есть на каждом слое конкатенировать выход из предыдущего слоя и информацию о классе.

In [ ]:
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()
    
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(7*7*32, 64),
            nn.ReLU(),
            nn.Linear(64, dim_code * 2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(dim_code, 64),
            nn.ReLU(),
            nn.Linear(64, 32*7*7),
            nn.Unflatten(1, torch.Size([32, 7, 7])),
            nn.Upsample(scale_factor=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 1, kernel_size=3, padding=1)
        )

    def encode(self, x, class_num):
        x = self.encoder(x).view(-1, 2, dim_code)
        
        mu = x[:, 0, :]
        logsigma = x[:, 1, :]
        
        return mu, logsigma, class_num
    
    def gaussian_sampler(self, mu, logsigma):
        if self.training:
            std = torch.exp(0.5 * logsigma)
            eps = torch.randn_like(std)
            sample = mu + (eps * std)
            return sample
        else:
            return mu
    
    def decode(self, z):       
        x = self.decoder(z)
        return x

    def forward(self, x, class_num):
        print(x.shape)
        print(class_num)
        
        mu, logsigma = self.encode(x, class_num)
        
        latent = self.gaussian_sampler(mu, logsigma)
            
        reconstruction = self.decode(latent)
        reconstruction = torch.sigmoid(reconstruction)
        
        return reconstruction, mu, logsigma

In [ ]:
criterion = loss_vae

autoencoder = CVAE().to(device)

optimizer = torch.optim.Adam(autoencoder.parameters())

In [ ]:
from tqdm.notebook import trange, tqdm

n_epochs = 10
train_losses = []
val_losses = []

for epoch in trange(n_epochs):
    autoencoder.train()
    train_losses_per_epoch = []
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        reconstruction, mu, logsigma = autoencoder(X_batch.to(device), y_batch)
        reconstruction = reconstruction.view(-1, 1, 28, 28)
        loss = criterion(X_batch.to(device).float(), mu, logsigma, reconstruction)
        loss.backward()
        optimizer.step()
        train_losses_per_epoch.append(loss.item())

    train_losses.append(np.mean(train_losses_per_epoch))

    autoencoder.eval()
    val_losses_per_epoch = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
          reconstruction, mu, logsigma = autoencoder(X_batch.to(device), y_batch)
          reconstruction = reconstruction.view(-1, 1, 28, 28)
          loss = criterion(X_batch.to(device).float(), mu, logsigma, reconstruction)
          val_losses_per_epoch.append(loss.item())

    val_losses.append(np.mean(val_losses_per_epoch))

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.arange(len(train_losses)), train_losses, label='Train')
plt.plot(np.arange(len(val_losses)), val_losses, label='Validation')

plt.xlabel('Epoch')
plt.title('MSE loss')
plt.legend()
plt.show()

In [ ]:
autoencoder.eval()
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        reconstruction, mu, logsigma = autoencoder(X_batch.to(device))
        reconstruction = reconstruction.view(-1, 28, 28, 1)
        result = reconstruction.cpu().detach().numpy()
        ground_truth = X_batch.numpy().transpose(0, 2, 3, 1)
        break

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 20))
for i, (gt, res) in enumerate(zip(ground_truth[:5], result[:5])):
  plt.subplot(5, 2, 2*i+1)
  plt.imshow(gt)
  plt.subplot(5, 2, 2*i+2)
  plt.imshow(res)

### Sampling


Тут мы будем сэмплировать из CVAE. Это прикольнее, чем сэмплировать из простого AE/VAE: тут можно взять один и тот же латентный вектор и попросить CVAE восстановить из него картинки разных классов!
Для MNIST вы можете попросить CVAE восстановить из одного латентного вектора, например, картинки цифры 5 и 7.

In [ ]:
<тут нужно научиться сэмплировать из декодера цифры определенного класса>

Splendid! Вы великолепны!


### Latent Representations

Давайте посмотрим, как выглядит латентное пространство картинок в CVAE и сравним с картинкой для VAE =)

Опять же, нужно покрасить точки в разные цвета в зависимости от класса.

In [ ]:
<ваш код получения латентных представлений, применения TSNE и визуализации>

Что вы думаете насчет этой картинки? Отличается от картинки для VAE?

# BONUS 1: Denoising

## Внимание! За бонусы доп. баллы не ставятся, но вы можете сделать их для себя.

У автоэнкодеров, кроме сжатия и генерации изображений, есть другие практические применения. Про одно из них эта бонусная часть задания.

Автоэнкодеры могут быть использованы для избавления от шума на фотографиях (denoising). Для этого их нужно обучить специальным образом: input картинка будет зашумленной, а выдавать автоэнкодер должен будет картинку без шума. 
То есть, loss-функция AE останется той же (MSE между реальной картинкой и выданной), а на вход автоэнкодеру будет подаваться зашумленная картинка.

<a href="https://ibb.co/YbRJ1nZ"><img src="https://i.ibb.co/0QD164t/Screen-Shot-2020-06-04-at-4-49-50-PM.png" alt="Screen-Shot-2020-06-04-at-4-49-50-PM" border="0"></a>

Для этого нужно взять ваш любимый датасет (датасет лиц из первой части этого задания или любой другой) и сделать копию этого датасета с шумом. 

В питоне шум можно добавить так:

In [ ]:
noise_factor = 0.5
X_noisy = X + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=X.shape) 

In [ ]:
<тут ваш код обучения автоэнкодера на зашумленных картинках. Не забудтье разбить на train/test!>

In [ ]:
<тут проверка, как AE убирает щум с тестовых картинок. Надеюсь, все получилось =)>

# BONUS 2: Image Retrieval

## Внимание! За бонусы доп. баллы не ставятся, но вы можете сделать их для себя.

Давайте представим, что весь наш тренировочный датасет -- это большая база данных людей. И вот мы получили картинку лица какого-то человека с уличной камеры наблюдения (у нас это картинка из тестового датасета) и хотим понять, что это за человек. Что нам делать? Правильно -- берем наш VAE, кодируем картинку в латентное представление и ищем среди латентныз представлений лиц нашей базы самые ближайшие!

План:

1. Получаем латентные представления всех лиц тренировочного датасета
2. Обучаем на них LSHForest `(sklearn.neighbors.LSHForest)`, например, с `n_estimators=50`
3. Берем картинку из тестового датасета, с помощью VAE получаем ее латентный вектор
4. Ищем с помощью обученного LSHForest ближайшие из латентных представлений тренировочной базы
5. Находим лица тренировочного датасета, которым соответствуют ближайшие латентные представления, визуализируем!

Немного кода вам в помощь: (feel free to delete everything and write your own)

In [ ]:
codes = <поучите латентные представления картинок из трейна>

In [ ]:
# обучаем LSHForest
from sklearn.neighbors import LSHForest
lshf = LSHForest(n_estimators=50).fit(codes)

In [ ]:
def get_similar(image, n_neighbors=5):
  # функция, которая берет тестовый image и с помощью метода kneighbours у LSHForest ищет ближайшие векторы
  # прогоняет векторы через декодер и получает картинки ближайших людей

  code = <получение латентного представления image>
    
  (distances,),(idx,) = lshf.kneighbors(code, n_neighbors=n_neighbors)

  return distances, X_train[idx]

In [ ]:
def show_similar(image):

  # функция, которая принимает тестовый image, ищет ближайшие к нему и визуализирует результат
    
    distances,neighbors = get_similar(image,n_neighbors=11)
    
    plt.figure(figsize=[8,6])
    plt.subplot(3,4,1)
    plt.imshow(image.cpu().numpy().transpose([1,2,0]))
    plt.title("Original image")
    
    for i in range(11):
        plt.subplot(3,4,i+2)
        plt.imshow(neighbors[i].cpu().numpy().transpose([1,2,0]))
        plt.title("Dist=%.3f"%distances[i])
    plt.show()

In [ ]:
<тут выведите самые похожие лица к какому-нибудь лицу из тестовой части датасета>